In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

import re
import glob
import csv

import time

import nltk

from sklearn.externals import joblib

In [12]:
df = pd.read_csv('data/MTStalk.csv')
print(df.head())

             id  conversation_id    created_at     date      time timezone  \
0  1.111440e+18     1.111440e+18  1.553820e+12  3/28/19  21:10:58      EDT   
1  1.111430e+18     1.111430e+18  1.553820e+12  3/28/19  21:04:26      EDT   
2  1.111430e+18     1.111430e+18  1.553820e+12  3/28/19  21:01:42      EDT   
3  1.111430e+18     1.111430e+18  1.553820e+12  3/28/19  20:58:05      EDT   
4  1.111430e+18     1.111430e+18  1.553820e+12  3/28/19  20:54:39      EDT   

        user_id         username                               name  place  \
0  5.887706e+08         mclneuro                           McLNeuro    NaN   
1  3.311707e+09    evodevodeanna                        deanna دينا    NaN   
2  8.903280e+17  inorganicveggie  Pomarrosa es cinismo hecho mujer.    NaN   
3  3.053047e+09      carriejm007                  CJ McAdams MD PHD    NaN   
4  8.903280e+17  inorganicveggie  Pomarrosa es cinismo hecho mujer.    NaN   

         ...        retweet gif_url gif_thumb video_url video_

In [13]:
print(df.tail())

               id  conversation_id    created_at     date      time timezone  \
218  1.098730e+18     1.098730e+18  1.550790e+12  2/21/19  18:56:58      EDT   
219  1.098730e+18     1.098730e+18  1.550790e+12  2/21/19  18:40:26      EDT   
220  1.098690e+18     1.098690e+18  1.550780e+12  2/21/19  16:18:02      EDT   
221  1.098650e+18     1.098650e+18  1.550770e+12  2/21/19  13:28:41      EDT   
222  1.098640e+18     1.098640e+18  1.550770e+12  2/21/19  12:53:55      EDT   

          user_id        username                         name  place  \
218  5.887706e+08        mclneuro                     McLNeuro    NaN   
219  5.887706e+08        mclneuro                     McLNeuro    NaN   
220  5.887706e+08        mclneuro                     McLNeuro    NaN   
221  1.202387e+09         kkgonza  Kalynda Gonzales Stokes PhD    NaN   
222  5.587324e+08  amympalubinsky                        Dr. P    NaN   

           ...        retweet  \
218        ...            NaN   
219        ...

In [14]:
df.describe()

id  conversation_id    created_at       user_id  place  \
count  2.230000e+02     2.230000e+02  2.230000e+02  2.230000e+02    0.0   
mean   1.102447e+18     1.102444e+18  1.551679e+12  4.706334e+17    NaN   
std    4.169597e+15     4.172379e+15  9.942279e+08  4.932292e+17    NaN   
min    1.098640e+18     1.098640e+18  1.550770e+12  1.032801e+07    NaN   
25%    1.098750e+18     1.098740e+18  1.550800e+12  5.887706e+08    NaN   
50%    1.100920e+18     1.100920e+18  1.551310e+12  3.311707e+09    NaN   
75%    1.103830e+18     1.103830e+18  1.552010e+12  1.006020e+18    NaN   
max    1.111440e+18     1.111440e+18  1.553820e+12  1.054920e+18    NaN   

       replies_count  retweets_count  likes_count  location  retweet  \
count     223.000000      223.000000   223.000000       0.0      0.0   
mean        0.430493        2.304933     7.569507       NaN      NaN   
std         0.693186        4.391466    15.598665       NaN      NaN   
min         0.000000        0.000000     0.000000       NaN      NaN   
25%         0.000000        0.000000     2.000000       NaN      NaN   
50%         0.000000        1.000000     4.000000       NaN      NaN   
75%         1.000000        3.000000     8.000000       NaN      NaN   
max         4.000000       44.000000   168.000000       NaN      NaN   

       video_url  video_thumb  is_quote_status      quote_id  is_reply_to  \
count        0.0          0.0       223.000000  2.230000e+02    223.00000   
mean         NaN          NaN         0.076233  8.405641e+16      0.22870   
std          NaN          NaN         0.265968  2.932863e+17      0.42094   
min          NaN          NaN         0.000000  0.000000e+00      0.00000   
25%          NaN          NaN         0.000000  0.000000e+00      0.00000   
50%          NaN          NaN         0.000000  0.000000e+00      0.00000   
75%          NaN          NaN         0.000000  0.000000e+00      0.00000   
max          NaN          NaN         1.000000  1.111430e+18      1.00000   

       has_parent_tweet  
count         223.00000  
mean            0.22870  
std             0.42094  
min             0.00000  
25%             0.00000  
50%             0.00000  
75%             0.00000  
max             1.00000

In [15]:
df.dtypes

id                  float64
conversation_id     float64
created_at          float64
date                 object
time                 object
timezone             object
user_id             float64
username             object
name                 object
place               float64
tweet                object
tags                 object
replies              object
mentions             object
urls                 object
photos               object
replies_count         int64
retweets_count        int64
likes_count           int64
location            float64
hashtags             object
link                 object
retweet             float64
gif_url              object
gif_thumb            object
video_url           float64
video_thumb         float64
is_quote_status       int64
quote_id            float64
quote_url            object
is_reply_to           int64
has_parent_tweet      int64
dtype: object

In [16]:
print(df['tweet'])

0      We will be posting more comprehensive awesome ...
1      Male allies can use their privilege to advocat...
2      Mirror how upsetting the situation is. Be emot...
3      #MTSTalk One of the most important ways people...
4      Unfortunately, retaliation happens more freque...
5      What you can do as a workplace peer that’s als...
6      When I told my boss about what happened to me,...
7      Talking about workplace support @ana_safavi sa...
8      Agree - don't tell people what they "should ha...
9      Being supportive and wonderful takes many form...
10     “Why didn’t you say no?”  ...is a nonsensical ...
11     Learn to listen! Avoid behavioral explanations...
12     “Why didn’t you say no?”  Power dynamics are v...
13     #MTSTalk Don't add judgement or questioning wh...
14     @carriejm007 is talking about how important it...
15     Come join the wonderful @MeTooSTEM squad to ta...
16     '  https://www.thearc.org/sslpage.aspx?pid=245...
17     'RAINN maintains a datab

In [17]:
df['username'].value_counts()

metoostem          73
mclneuro           47
amympalubinsky     21
inorganicveggie    18
charter4values     15
brittenymwatson    10
profsharona         6
evodevodeanna       6
sciswany            4
joshfessel          4
johanna_folk        3
nudoces             3
kkgonza             3
minoritypostdoc     2
lenidiamond         1
fungalsquirrel      1
carriejm007         1
lindambui           1
hormiga             1
neeleyeunice        1
tanyadapkey         1
fellgernon          1
Name: username, dtype: int64

In [18]:
df['name'].value_counts()

MeTooSTEM                            73
McLNeuro                             47
Dr. P                                21
Pomarrosa es cinismo hecho mujer.    18
CharterForHealthcare                 15
Britteny Watson                      10
Dr. Sharona Gordon                    6
deanna دينا                           6
JoshFessel                            4
Dr. Teresa Swanson                    4
Erica Smith 🏳️‍🌈                      3
Kalynda Gonzales Stokes PhD           3
Johanna Folk                          3
Alberto Roca                          2
Berning For the Common Good           1
Terry McGlynn                         1
Dr. Linda Bui                         1
Eunice Neeley                         1
Debra DeLoach 🍄🐿️                     1
CJ McAdams MD PHD                     1
Tanya Dapkey 🐛🦋                       1
🇲🇽 Dr. Leonardo Collado-Torres        1
Name: name, dtype: int64

In [19]:
df['date'].value_counts()

2/21/19    74
3/7/19     56
2/27/19    50
3/28/19    29
2/22/19     6
3/14/19     4
3/21/19     2
3/1/19      1
3/24/19     1
Name: date, dtype: int64